# Prepare

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install paddlepaddle-gpu
!pip install paddleocr
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 89.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114

# path setting

In [3]:
import os
video_path = '/content/drive/MyDrive/SMP/clips/tof_1_bank_code_addr.mp4'  # Path to your video file
root_folder = '/content/drive/MyDrive/SMP' # where everything is placed
extracted_text_folder = os.path.join(root_folder, 'extracted_text') # where extracted text is
extracted_frame_folder = os.path.join(root_folder, 'extracted_frame') # where extracted frames are

if not os.path.exists(extracted_text_folder):
    os.makedirs(extracted_text_folder)
if not os.path.exists(extracted_frame_folder):
    os.makedirs(extracted_frame_folder)

# extract the name of video
# Extract the base name (filename with extension)
base_name_with_extension = os.path.basename(video_path)
# Remove the extension to get just the videoname
video_name = os.path.splitext(base_name_with_extension)[0]

frame_rate = 30 # how many frames per second

process_rate = 2 # how many frames to process per second
r = 3 # how many frames to process per second

# text extraction


*   video_path: which video to process
*   folder_name: should be same as the name of the video for project management purpose
* r: rate of processing, ex. 5 means every 5th frame is processed


OPENCV and paddle OCR to extract all text from video, save text to txt file for manual inspection,
and json file for sensitive frame identification

In [6]:
import cv2
import paddleocr
from paddleocr import PaddleOCR
import json
import os

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # You can specify the language

def extract_text_from_video(video_path, folder_name, r):
    # Ensure that folder_name is a valid directory name (by joining with global base path)
    output_folder = os.path.join(extracted_text_folder, folder_name)

    # Ensure the output folder exists, create it if necessary
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Define paths for the output files inside the new folder
    text_file_path = os.path.join(output_folder, 'extracted_text.txt')
    json_file_path = os.path.join(output_folder, 'extracted_text.json')

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Create a list to hold all the data for the frames (for JSON)
    video_data = []

    frame_num = 0

    # Open the text file for writing
    with open(text_file_path, 'w') as text_file:

        # Loop through each frame in the video
        while True:
            ret, frame = cap.read()  # Read the current frame
            if not ret:
                break  # Exit loop if no frame is returned (end of video)

            frame_num += 1  # Increment frame number

            # Only process every r-th frame
            if frame_num % r != 0:
                continue

            # Convert frame to grayscale for better OCR accuracy
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Use PaddleOCR to extract text and coordinates from the frame
            result = ocr.ocr(gray_frame)

            # Check if any text is detected in the frame
            if result:
                # Write the frame number in the text file
                text_file.write(f"Frame {frame_num}:\n")

                # Create a dictionary for this frame for JSON output
                frame_data = {"frame_number": frame_num, "texts": []}

                # Process OCR result to extract text and its coordinates
                for line in result:
                    for box in line:
                        # Extract coordinates (discard confidence score)
                        coordinates = box[0]  # List of four points [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
                        # Extracted text
                        extracted_text = box[1][0]

                        # Add the text and its coordinates to the frame data for JSON
                        frame_data["texts"].append({
                            "text": extracted_text,
                            "coordinates": coordinates
                        })

                        # Write the text in the text file (without coordinates)
                        text_file.write(f"{extracted_text}\n")

                # Append the frame's data to the video data list (for JSON)
                video_data.append(frame_data)

                # Write a new line to separate frame data in the text file
                text_file.write("\n")

    # Release the video capture object
    cap.release()

    # Save the extracted data to a JSON file (without confidence scores)
    with open(json_file_path, 'w') as json_file:
        json.dump(video_data, json_file, indent=4)

    print(f"Data has been written to {text_file_path} and {json_file_path}")

# Example usage
extract_text_from_video(video_path, video_name, r) # rate set to r, change it in path setting


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:00<00:00, 6144.56it/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:00<00:00, 12385.75it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:00<00:00, 3818.72it/s]

[2024/11/02 22:59:18] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_ch

[2024/11/02 22:59:19] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/11/02 22:59:21] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/11/02 22:59:26] ppocr DEBUG: dt_boxes num : 18, elapsed : 1.8450982570648193
[2024/11/02 22:59:26] ppocr DEBUG: cls num  : 18, elapsed : 0.13716888427734375
[2024/11/02 22:59:26] ppocr DEBUG: rec_res num  : 18, elapsed : 0.19122028350830078
[2024/11/02 22:59:26] ppocr DEBUG: dt_boxes num : 18, elapsed : 0.03725004196166992
[2024/11/02 22:59:26] ppocr DEBUG: cls num  : 18, elapsed : 0.041661739349365234
[2024/11/02 22:59:26] ppocr DEBUG: rec_res num  : 18, elapsed : 0.03391599655151367
[2024/11/02 22:59:26] ppocr DEBUG: dt_boxes num : 18, elapsed : 0.03176474571228027
[2024/11/02 22:59:26] ppocr DEBUG: cls num  : 18, elapsed : 0.025059223175048828
[2024/11/02 22:59:26] ppocr DEBUG: rec_res num  : 18, elapsed : 0.033875465393066406
[2024/11/02 22:59:26] ppocr DEBUG: dt_boxes num : 18, elapsed : 

# function to extract all frames from a video (for debugging purpose, not necessary for the acutal processing)
*   video_path: source of video
*   output_path: which folder to put the frames
*   folder_name: name of folder to put frames




In [ ]:
import cv2
import os

def extract_frames_from_video(video_path, video_name):
    # Create the full path for the folder where frames will be saved
    output_folder = os.path.join(extracted_frame_folder, video_name)

    # Ensure the output folder exists, create it if necessary
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_num = 0  # Frame counter

    # Loop through each frame of the video
    while True:
        ret, frame = cap.read()  # Read the current frame
        if not ret:
            break  # Exit loop if no frame is returned (end of video)

        # Generate the output file name for each frame
        frame_filename = os.path.join(output_folder, f"frame_{frame_num:04d}.png")

        # Save the frame as an image file
        cv2.imwrite(frame_filename, frame)

        # Print progress for debugging
        print(f"Extracted: {frame_filename}")

        frame_num += 1  # Increment frame counter

    # Release the video capture object
    cap.release()

    print(f"Extraction complete. {frame_num} frames saved to {output_folder}.")

# Example usage
#extract_frames_from_video(video_path, video_name)

Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0000.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0001.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0002.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0003.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0004.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0005.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0006.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0007.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0008.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0009.png
Extracted: /content/drive/MyDrive/SMP/extracted_frame/tof_1_bank_code_addr/frame_0010.png
Extracted:

# regular expression for sensetive information labeling
this keep what format of data will be identified as sensitive information, add more rows to filter out the desired text

In [7]:
import re

def match_sensitive_info(text):
    # Define regular expressions for each type of information
    patterns = {
        "myGov username": r"\b[A-Z0-9]{8}\b",  # Matches alphanumeric myGov username
        "Date of birth": r"\b\d{1,2}\s(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}\b",  # Matches date format '2 March 1986'
        "email": r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",  # Matches email addresses
        "code": r"\b\d{6}\b",  # Matches 6-digit codes
        "Individual Healthcare Identifier": r"\b\d{16}\b",  # Matches 16-digit Healthcare Identifier
        "Phone numbers": r"\b0\d{9}\b",  # Matches Australian mobile phone numbers starting with '0'
        "BSB/Account": r'\d{3}-\d{3}|\d{7}',
        "card number": r'\b(?:\d[ -]*?){13,19}\b',
        'cvc': r'\b\d{3,4}\b'

    }

    # Loop over each pattern and see if it matches the text
    for label, pattern in patterns.items():
        # Find the position of the ':' symbol and extract the text after it
        #if f"{label}:" in text:
         #   after_colon_text = text.split(f"{label}:")[-1].strip()
        #if re.match(pattern, after_colon_text):
        if re.search(pattern, text):
            return True

    return False




# test case for regex (debugging purpose)
also test performance difference between re.search and match

In [ ]:
# Example usages
test_cases = [
    'myGov username: BN807751',
    'Date of birth: 2 March 1986',
    'email: karan.nagrani86@gmail.com',
    'code: 474191',
    'Individual Healthcare Identifier: 8003601233784268',
    'Phone numbers Mobile: 0430954922',
    'BSB/Account: 123-456',
]

# Test the function with sample inputs
#for case in test_cases:
#    print(f"{case}: {match_sensitive_info(case)}")

myGov username: BN807751: True
Date of birth: 2 March 1986: True
email: karan.nagrani86@gmail.com: True
code: 474191: True
Individual Healthcare Identifier: 8003601233784268: True
Phone numbers Mobile: 0430954922: True
BSB/Account: 123-456: True


In [ ]:
import re
import time

# Test string
text = "a" * 1000000 + "pattern"

# Pattern to match
pattern = r"pattern"

# Using re.match() (This will not find the pattern because it is not at the start)
start_time = time.time()
match = re.match(pattern, text)
print(f"re.match() took {time.time() - start_time:.6f} seconds")

# Using re.search() (This will find the pattern anywhere in the string)
start_time = time.time()
search = re.search(pattern, text)
print(f"re.search() took {time.time() - start_time:.6f} seconds")

# about 6x times slower

re.match() took 0.000181 seconds
re.search() took 0.001135 seconds


# function to remove consecutive repeated frames
* name: name of the file that contains the extracted text
remove all identical frames from the extracted_text.txt, keep only the first frame and record the number of frame thats are same, use that information to process the json file and remove identical frames too, keeps the first frame and add a new data "repeated_number" along with the "frame_number" and "text". \
Although it works for now, this function need more testing and evaluation on it, the text content might not changing the position of each text might changed between two sampled frame, maybe a positon prediction is needed to make blocking to track the target

In [8]:
import os
import json

def remove_consecutive_identical_frames_from_json(name):
    # Path for the JSON file
    json_file_path = os.path.join(extracted_text_folder, name+"/extracted_text.json")

    # Load the JSON file content
    with open(json_file_path, 'r') as json_file:
        json_data = json.load(json_file)

    # New list to store unique frames in JSON
    unique_json_data = []
    previous_frame_text = None
    previous_frame_data = None

    for frame_data in json_data:
        # Concatenate the text fields into a single string for comparison
        frame_text = " ".join([text['text'] for text in frame_data['texts']])

        if frame_text == previous_frame_text:
            # Consecutive identical frame, increment the count
            previous_frame_data["repeated_number"] += 1
        else:
            # If there was a previous frame, add it to the list of unique frames
            if previous_frame_data:
                unique_json_data.append(previous_frame_data)

            # Start tracking the new frame
            previous_frame_data = frame_data
            previous_frame_data["repeated_number"] = 1  # Initialize the repeated number
            previous_frame_text = frame_text

    # Add the last frame if it hasn't been added
    if previous_frame_data:
        unique_json_data.append(previous_frame_data)

    # put it in the same folder
    output_path = os.path.join(extracted_text_folder, name+"/extracted_text_unique.json")

    # Write back the unique JSON data with repetition info
    with open(output_path, 'w') as output_json_file:
        json.dump(unique_json_data, output_json_file, indent=4)

    print(f"Processed {json_file_path}, unique JSON file saved as {output_path}.")

# Example usage
#file_name = 'extracted_text_with_coordinates'  # Assuming your file is 'extracted_text_with_coordinates.json'
remove_consecutive_identical_frames_from_json(video_name)



Processed /content/drive/MyDrive/SMP/extracted_text/tof_1_bank_code_addr/extracted_text.json, unique JSON file saved as /content/drive/MyDrive/SMP/extracted_text/tof_1_bank_code_addr/extracted_text_unique.json.


# function to label the sensitive word
* name: where the unique text json is hold
this function use function match_sensitive_info to find the sensitive words in the json file, and set the "is_sensitive" of that text to true, after that all the sensitive words will be parsed into its incomple form, ex. "abc" will be parsed with "ab" and "a", and this bag of words will be returned as a dictionary.

In [9]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()
    """
    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True
    """
    # Insert both the original word and the reversed word
    def insert(self, word):
        # Insert the original word
        self._insert_word(word)

        # Insert the reversed word
        reversed_word = word[::-1]
        self._insert_word(reversed_word)

    # Private method to insert a word into the Trie
    def _insert_word(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def search(self, word):
        if len(word) == 1:
            return False
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

    # Reverse search: search for the reversed input text in the Trie
    def reverse_search(self, word):
        reversed_word = word[::-1]  # Reverse the input word
        return self.search(reversed_word)

    def partial_match(self, word):
        if self.search(word) or self.reverse_search(word):
            return True
        return False

# dont use search_substring, it is not working properly
"""
    def search_substring(self, word):
        node = self.root
        for i in range(len(word)):
            if self.search(word[i:]):
                return True
        return False
"""

"""
# List of strings
strings = ["abcd", "catatat", "doododog"]

# Build the Trie
trie = Trie()

for s in strings:
    trie.insert(s)

# Check if substrings exist in any of the strings
substrings_to_check = ["bcd", 'a', 'd', 'ab', "aetate", "tata", "do", "dog"]

# Search for each substring
for s in substrings_to_check:
    result = trie.partial_match(s)
    print(f"'{s}' found in any string? {result}")
"""


'\n# List of strings\nstrings = ["abcd", "catatat", "doododog"]\n\n# Build the Trie\ntrie = Trie()\n\nfor s in strings:\n    trie.insert(s)\n\n# Check if substrings exist in any of the strings\nsubstrings_to_check = ["bcd", \'a\', \'d\', \'ab\', "aetate", "tata", "do", "dog"]\n\n# Search for each substring\nfor s in substrings_to_check:\n    result = trie.partial_match(s)\n    print(f"\'{s}\' found in any string? {result}")\n'

## label incomplet form of sensitive info
* words_set: the unique sensitive word set
* name: name of the video we are processing
this function check the labled_extracted_text.json and label any incomplete form of sensitive info, so that no information is leaked before inputting complete

In [10]:
def unique_sensitive_word_set(name): # returns a set of unique sensitive words
    # Path to the JSON file holding the unique text
    json_file_path = os.path.join(extracted_text_folder, name + "/extracted_text_unique.json")

    # Load the JSON file
    with open(json_file_path, 'r') as json_file:
        json_data = json.load(json_file)

    sensitive_words_set = set()  # Use a set to store unique sensitive words

    # Loop through each frame in the JSON
    for frame_data in json_data:
        for text_data in frame_data['texts']:
            text = text_data['text']

            # Use match_sensitive_info to detect if the text contains sensitive information
            if match_sensitive_info(text):
                # Set the "is_sensitive" flag to True
                text_data['is_sensitive'] = True

                # Add the full text to the set (ensuring uniqueness)
                sensitive_words_set.add(text)
            else:
                text_data['is_sensitive'] = False



    # label incomplete words
    trie = Trie()
    for string in sensitive_words_set:
        trie.insert(string)
    """
    for string in sensitive_words_set:
        for i in range(len(string)):
            trie.insert(string[i:])
    """

    # Loop through the JSON to check and label incomplete forms
    for frame_data in json_data:
        for text_data in frame_data['texts']:
            text = text_data['text']

            # Check if any part of the text matches incomplete sensitive info
            if trie.partial_match(text):
                # Set the "is_sensitive" flag to True, partial match
                text_data['is_sensitive'] = True
            else:
                text_data['is_sensitive'] = False


    # Save the updated JSON with "is_sensitive" flags
    output_json_path = os.path.join(extracted_text_folder, name + "/labled_extracted_text.json")
    with open(output_json_path, 'w') as output_json_file:
        json.dump(json_data, output_json_file, indent=4)

    # Return the list of unique sensitive words
    return sensitive_words_set

# Example usage
unique_sensitive_words = unique_sensitive_word_set(video_name)
print(unique_sensitive_words)


{'1832', 'Linked15Jul 2014', '000000', '1831', 'Linked14Feb 2018', '2March 1986', 'Linked 14 Feb 2018', '3335', 'Linked 19Nov 2019', 'mobile number XXXXXXX 922', '2 March 1986', 'AWIHURIN CAO', 'Use "karan.n...6@gmail.com\'', 'Use "karan.n...6@gmail.com"', 'mobile number XXXX XXX 922.', '8003601233784268', 'Last Updated1November 2023', 'VIC 3123', '003', "6@gmail.com'", 'Use"karan.n...6@gmail.com\'', 'mobile number XxxxXxx 922.', '2 March 198', '0430954922', 'Last Updated 1 November 2023', 'Digital Identity on 8 Jul 2023', '0401446', 'Linked 1 Dec 2017', 'ULTIMATE', 'KARAN.NAGRANI86@GMAIL.COM', '150', 'mobile number XxXXXXx 922', 'mobile number XXXX XXx 922', 'HAWTHORN EAST', 'karan.nagrani86@gmail.com', 'Linked 14Feb 2018', '474119', 'Linked 15Jul 2014', 'Last login 16 April 2024 at 9:08 AM AEST', 'Digital ldentity on 8 Jul 2023', 'mobile number Xxxxxxx 922', 'Linked19 Nov 2019', 'Use "karan.n...6@gmail.com\'.', '11/4/2024', 'Linked 15 Jul 2014', '1834', '306-065', 'Last Updated 1Nove

experiment on trie, currently it can match "a" "ab" "abc" to "abcd", but not "bcd"

In [11]:
"""
# label incomplete words
trie = Trie()
# Build the Trie by inserting only full sensitive words
for string in unique_sensitive_words:
    trie.insert(string)

# Search for exact match or meaningful substrings
text = "November 2023"
#if trie.search(text):  # Only match the full text or valid sensitive word
if trie.search_substring(text):  # Only match the full text or valid sensitive word
    print("true")
else:
    print("false")
"""

'\n# label incomplete words\ntrie = Trie()\n# Build the Trie by inserting only full sensitive words\nfor string in unique_sensitive_words:\n    trie.insert(string)\n\n# Search for exact match or meaningful substrings\ntext = "November 2023"\n#if trie.search(text):  # Only match the full text or valid sensitive word\nif trie.search_substring(text):  # Only match the full text or valid sensitive word\n    print("true")\nelse:\n    print("false")\n'

# function to block the sensitive information in video
* name: the name of the input video

using the labeled_extracted_text.json to place blocking box in the video, using the coordinate of text that has its "is_sensitive" equals to true, if there is text been labeled as sensitive in the frame number n, using the coordinate from the current frame and the repeated frame number k to placing blocking box in the following frame number n-4 to n+5*(k-1)+4

In [12]:
import os
import cv2
import json

def block_sensitive_info_in_video(name):
    # Path to the labeled JSON file
    json_file_path = os.path.join(extracted_text_folder, name + "/labled_extracted_text.json")
    #json_file_path = /content/drive/MyDrive/SMP/extracted_text/tof_1_bank_code_addr/labled_extracted_text.json

    # Path to the video file
    video_file_path = video_path

    # Load the JSON file
    with open(json_file_path, 'r') as json_file:
        json_data = json.load(json_file)

    # Open the video file
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_file_path}")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Output video with blocked sensitive information
    #output_video_path = os.path.join(extracted_text_folder, f"{name}/blocked_video.mp4")
    output_video_path = '/content/drive/MyDrive/SMP/result/blocked_video.mp4'
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Create a mapping from frame number to its sensitive regions
    frame_sensitive_boxes = {}

    for frame_data in json_data:
        frame_number = frame_data["frame_number"]
        repeated_number = frame_data.get("repeated_number", 1)  # Default repeated number to 1
        start_frame = max(0, frame_number - (r-1))  # Start frame for blocking, can't be negative
        end_frame = min(total_frames - 1, frame_number + r*(repeated_number - 1) + (r-1))  # End frame for blocking

        for text_data in frame_data['texts']:
            if text_data.get('is_sensitive', False):
                coordinates = text_data['coordinates']  # Get the coordinates of the sensitive text

                # Add the coordinates to every frame from start_frame to end_frame
                for frame in range(start_frame, end_frame + 1):
                    if frame not in frame_sensitive_boxes:
                        frame_sensitive_boxes[frame] = []
                    frame_sensitive_boxes[frame].append(coordinates)


    print(frame_sensitive_boxes)

    # Process each frame and block sensitive info
    frame_num = 0
    while True:
        ret, frame = cap.read()  # Read the current frame
        if not ret:
            break  # End of video

        # Check if the current frame contains any sensitive information
        if frame_num in frame_sensitive_boxes:
            for coordinates in frame_sensitive_boxes[frame_num]:
                # Draw a black rectangle over the sensitive area
                # Coordinates: [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]
                x1, y1 = coordinates[0]
                x2, y2 = coordinates[2]  # Assuming top-left to bottom-right coordinates

                # Draw the blocking rectangle on the frame
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 0), -1)

        # Write the frame to the output video
        out.write(frame)

        frame_num += 1

    # Release video capture and output
    cap.release()
    out.release()

    print(f"Blocked sensitive information saved to {output_video_path}")

# Example usage
block_sensitive_info_in_video(video_name)


{145: [[[72.0, 638.0], [229.0, 641.0], [229.0, 663.0], [72.0, 660.0]], [[229.0, 679.0], [383.0, 675.0], [384.0, 701.0], [230.0, 706.0]]], 146: [[[72.0, 638.0], [229.0, 641.0], [229.0, 663.0], [72.0, 660.0]], [[229.0, 679.0], [383.0, 675.0], [384.0, 701.0], [230.0, 706.0]]], 147: [[[72.0, 638.0], [229.0, 641.0], [229.0, 663.0], [72.0, 660.0]], [[229.0, 679.0], [383.0, 675.0], [384.0, 701.0], [230.0, 706.0]]], 148: [[[72.0, 638.0], [229.0, 641.0], [229.0, 663.0], [72.0, 660.0]], [[229.0, 679.0], [383.0, 675.0], [384.0, 701.0], [230.0, 706.0]], [[72.0, 610.0], [384.0, 609.0], [384.0, 631.0], [72.0, 633.0]]], 149: [[[72.0, 638.0], [229.0, 641.0], [229.0, 663.0], [72.0, 660.0]], [[229.0, 679.0], [383.0, 675.0], [384.0, 701.0], [230.0, 706.0]], [[72.0, 610.0], [384.0, 609.0], [384.0, 631.0], [72.0, 633.0]]], 150: [[[72.0, 610.0], [384.0, 609.0], [384.0, 631.0], [72.0, 633.0]]], 151: [[[72.0, 610.0], [384.0, 609.0], [384.0, 631.0], [72.0, 633.0]], [[71.0, 600.0], [383.0, 601.0], [383.0, 627.0